<a href="https://colab.research.google.com/github/Kaleab1999/Medical-Transcription-for-medicne-department/blob/main/Bio_clinicalbert_for_MT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
import re
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, Trainer, TrainingArguments
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
df= pd.read_csv("/content/preproceed MT file.csv")
df.head()

,Unnamed: 0,transcription,medical_specialty,tokenised,stemmed,stemmed_without_stop
0,3,d m mode left atrial enlargement with left a...,Cardiovascular / Pulmonary,"['d', 'm', 'mode', 'left', 'atrial', 'enlargem...","['d', 'm', 'mode', 'left', 'atrial', 'enlarg',...",mode left atrial enlarg left atrial diamet cm ...
1,4,the left ventricular cavity size and wall thi...,Cardiovascular / Pulmonary,"['the', 'left', 'ventricular', 'cavity', 'size...","['the', 'left', 'ventricular', 'caviti', 'size...",left ventricular caviti size wall thick appear...
2,7,d echocardiogram multiple views of the heart ...,Cardiovascular / Pulmonary,"['d', 'echocardiogram', 'multiple', 'views', '...","['d', 'echocardiogram', 'multipl', 'view', 'of...",echocardiogram multipl view heart great vessel...
3,9,description normal cardiac chambers size nor...,Cardiovascular / Pulmonary,"['description', 'normal', 'cardiac', 'chambers...","['descript', 'normal', 'cardiac', 'chamber', '...",descript normal cardiac chamber size normal le...
4,11,d study mild aortic stenosis widely calcifie...,Cardiovascular / Pulmonary,"['d', 'study', 'mild', 'aortic', 'stenosis', '...","['d', 'studi', 'mild', 'aortic', 'stenosi', 'w...",studi mild aortic stenosi wide calcifi minim r...


In [ ]:
model_name = 'emilyalsentzer/Bio_ClinicalBERT'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model

In [ ]:
df['medical_specialty'].value_counts()

 Cardiovascular / Pulmonary    371
 General Medicine              259
 Gastroenterology              224
 Neurology                     223
Name: medical_specialty, dtype: int64

In [ ]:
df.shape

(1077, 6)

In [ ]:
data= df[['stemmed_without_stop','medical_specialty']]
data.head()

,stemmed_without_stop,medical_specialty
0,mode left atrial enlarg left atrial diamet cm ...,Cardiovascular / Pulmonary
1,left ventricular caviti size wall thick appear...,Cardiovascular / Pulmonary
2,echocardiogram multipl view heart great vessel...,Cardiovascular / Pulmonary
3,descript normal cardiac chamber size normal le...,Cardiovascular / Pulmonary
4,studi mild aortic stenosi wide calcifi minim r...,Cardiovascular / Pulmonary


In [ ]:
data.isnull().sum()

stemmed_without_stop    0
medical_specialty       0
dtype: int64

In [ ]:
data.iloc[[6]]

,stemmed_without_stop,medical_specialty
6,histori present ill patient year old femal pre...,General Medicine


In [ ]:
X= data['stemmed_without_stop'].tolist()
y= list( data['medical_specialty'].astype('category').cat.codes)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2)

### model building

In [ ]:
train_d= tokenizer(X_train, padding= True, truncation= True, max_length=512)
test_d= tokenizer(X_test, padding= True, truncation= True, max_length=512)

In [ ]:
train_d.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

#### create dataset

In [ ]:
import torch

In [ ]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
train_dataset = Dataset(train_d, y_train)
test_dataset = Dataset(test_d, y_test)

In [ ]:
train_dataset[5]

{'input_ids': tensor([  101,  2548,  2812,  5351,  2052,  4321,  1643,  1233,  1110,  6385,
          1113,  1665,  3689, 10272, 23123,  2980, 19700,  8362,  1643, 12316,
          5926,  1884, 10161,  7126,  1316,  5351,  1227,   171,  7880, 22593,
          7903,  1775,  7299,  1884, 10161,  7126,  1129,  2599,  1361,  1120,
         13119, 20497, 27647,  1233,  6625,  1248,  1159,  2951, 19814, 10272,
         23123,  2980, 19700,  2370,  1145,  2204,  1989,  2403, 19073, 11259,
          5559,  2489,  3420,  2489,  1243,   193,  7586,  5110,  5656,  1126,
          1377,  1233,  2330,  2951, 19814,  1260,  4915,  4267, 19885,  1116,
          1298,   175, 19366, 20362,  3805,  1293,  6348,  1464,  2489,  3073,
          6154, 14516,  4121,  2440,   192,  3864,   182,  8456, 11850,  1361,
          8255,  4608,  1243, 21447, 21718,  2836, 13477,  2037,  1884, 10161,
          7126,  6625,  3187,  3576,  1383,  1120, 13119, 20497, 27647,  1233,
           185,  7889,  5053,  1665, 12

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np

def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    # average= 'micro' because it's multi-class classification u can use also macro, or weight or other 
    # u shouldn't mention anything for binary classification
    recall = recall_score(y_true=labels, y_pred=pred, average='micro')
    precision = precision_score(y_true=labels, y_pred=pred, average='micro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='micro')

    return { "precision": precision, "recall": recall, 'f1-score': f1, "acc": accuracy}

In [ ]:
# Define Trainer
args = TrainingArguments(
    output_dir="resl",
    num_train_epochs=5,
    per_device_train_batch_size=16

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 861
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 270
  Number of trainable parameters = 108313348


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=270, training_loss=0.15202922114619502, metrics={'train_runtime': 427.528, 'train_samples_per_second': 10.07, 'train_steps_per_second': 0.632, 'total_flos': 1132713433313280.0, 'train_loss': 0.15202922114619502, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 216
  Batch size = 8


<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.6295365691184998,
 'eval_precision': 0.8518518518518519,
 'eval_recall': 0.8518518518518519,
 'eval_f1-score': 0.8518518518518519,
 'eval_acc': 0.8518518518518519,
 'eval_runtime': 7.4003,
 'eval_samples_per_second': 29.188,
 'eval_steps_per_second': 3.648,
 'epoch': 5.0}